In [1]:
import transformers
import datasets
import torch
from nltk import sent_tokenize
from helper_functions import _prepare_inputs_for_ans_extraction, _prepare_inputs_for_qg_from_answers_hl, get_sent_answer_list

In [2]:
ans_ext_model = transformers.AutoModelForSeq2SeqLM.from_pretrained('output_models/model_ans_ext_hl_t5')
qg_model = transformers.AutoModelForSeq2SeqLM.from_pretrained('output_models/model_qg_hl_t5')
tokenizer = transformers.AutoTokenizer.from_pretrained('tokenizers/t5_qg_tokenizer')

In [3]:
import gradio as gr

def generate_answer_question_texts(text):
    sents, answers = get_sent_answer_list(text, ans_ext_model, tokenizer)

    ans_output_text = ""
    for i, ans in enumerate(answers):
        ans_output_text += "{}: {}\n".format(i+1, ans)
        # print(output_texts)
    
    # print(output_texts)

    inputs = _prepare_inputs_for_qg_from_answers_hl(sents, answers)
    qg_encodings = tokenizer.batch_encode_plus(inputs, padding=True)

    output_ids = qg_model.generate(
        torch.tensor(qg_encodings['input_ids']),
        attention_mask = torch.tensor(qg_encodings['attention_mask']),
        max_length = 32
    )

    question_texts = tokenizer.batch_decode(
        output_ids, skip_special_tokens=True,clean_up_tokenization_spaces=True)

    question_output_text = ''
    for i, sent in enumerate(question_texts):
        question_output_text += "{}: {}\n".format(i+1, sent)
        # print(output_texts)
    return ans_output_text, question_output_text



context_in = gr.inputs.Textbox(lines=20, label='Context')
answers_out = gr.outputs.Textbox()
question_out = gr.outputs.Textbox()

demo = gr.Interface(
    generate_answer_question_texts,
    context_in,
    [answers_out,question_out],
    theme='huggingface',
    layout='horizontal',
    title="Answer extraction and question generation based on T5")

demo.launch(share=True)

Running locally at: http://127.0.0.1:7864/
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://39595.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7864/',
 'https://39595.gradio.app')